# 1.0 Get Data Of Stock Market

* 本篇代码用来查询所有股票数据，所有导出的数据都将导出到stockdata文件夹内，方便管理查看。
* 平台api方面，国内用tushare，国外用的雅虎。
* 本篇代码也计划用来做数据清洗并且数据展示，处理好的文件也存在stockdata文件夹里面。

In [ ]:
print("Hello,stock market")
from datetime import datetime
start_time = datetime.now()
import time
import numpy as np
## todo: calculating the time of import libraries
#先引入后面分析、可视化等可能用到的库
import tushare as ts
import pandas as pd
import matplotlib.pyplot as plt
#正常显示画图时出现的中文和负号
from pylab import mpl
import yfinance as yf
import matplotlib.dates as mdates
import seaborn as sns
# todo: solve chinese problem for plt
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

#显示所有列
# pd.set_option('display.max_columns', 100)
# #显示所有行
# pd.set_option('display.max_rows', 100)
# #设置value的显示长度为100，默认为50
# pd.set_option('max_colwidth',100)
print("Load Time: ",datetime.now()-start_time)

## 1.1 use tushare api get data

In [ ]:
# login tushare, use token
pro = ts.pro_api('f3987990df2a25038e5123dd321b5ed941048135b923e9c6415f20e1')

def getStockCompanyInformation(status='L', stock_code=''):
    """
    得到目前所有的股票公司信息
    :param status: 上市状态 L上市 D退市 P暂停上市，默认是L
    :param stock_code: TS股票代码
    :return:
    """

    if stock_code== '':
        rawStockCompanyDataInTushare = pro.stock_basic(exchange='', list_status=status, fields='ts_code,symbol,fullname,enname,name,area,industry,list_date')
    else:
        rawStockCompanyDataInTushare = pro.stock_basic(ts_code=stock_code, exchange='', list_status=status, fields='ts_code,symbol,fullname,enname,name,area,industry,list_date')

    rawStockCompanyDataInTushare['ts_code']=rawStockCompanyDataInTushare['ts_code'].astype(str)
    rawStockCompanyDataInTushare['symbol']=rawStockCompanyDataInTushare['symbol'].astype('float64')
    rawStockCompanyDataInTushare['name']=rawStockCompanyDataInTushare['name'].astype(str)
    rawStockCompanyDataInTushare['area']=rawStockCompanyDataInTushare['area'].astype(str)
    rawStockCompanyDataInTushare['industry']=rawStockCompanyDataInTushare['industry'].astype(str)
    rawStockCompanyDataInTushare['fullname']=rawStockCompanyDataInTushare['fullname'].astype(str)
    rawStockCompanyDataInTushare['enname']=rawStockCompanyDataInTushare['enname'].astype(str)
    rawStockCompanyDataInTushare['list_date']=rawStockCompanyDataInTushare['list_date'].astype('float64')

    rawStockCompanyDataInTushare['list_date']=pd.to_datetime(rawStockCompanyDataInTushare['list_date'], format='%Y%m%d')
    return rawStockCompanyDataInTushare


# 拉取数据
def getDailyStockData(stock_code="",start="20130101",end="20230101"):
    """
    拉取股票日结交易信息，时间段默认 2018-1-1 到 2022-1-1，注意：该函数单次只能查询6000条记录。
    :param stock_code: 股票代码，默认为空，输入为空的时候，拉取这段时期所有股票的交易消息；若不为空，拉去这只股票这段时间的交易消息
    :param start: 开始日期
    :param end: 结束日期
    :return:
    """
    rawData = pro.daily(**{
        "ts_code": stock_code,
        "trade_date": "",
        "start_date": start,
        "end_date": end,
        "offset": "",
        "limit": "",
    }, fields=[
        "ts_code",
        "trade_date",
        "open",
        "high",
        "low",
        "close",
        "pre_close",
        "change",
        "pct_chg",
        "vol",
        "amount"
    ])
    rawData['ts_code']=rawData['ts_code'].astype(str)
    rawData['trade_date']=rawData['trade_date'].astype('float64')

    rawData['trade_date']=pd.to_datetime(rawData['trade_date'], format='%Y%m%d')
    return rawData

In [ ]:
# rawCompanyDataInTushare=getStockCompanyInformation()
# rawCompanyDataInTushare.to_csv('stockData/rawCompanyDataInTushare.csv',header=rawCompanyDataInTushare.columns,index=False)

rawCompanyDataInTushare=pd.read_csv('stockData/rawCompanyDataInTushare.csv')
rawCompanyDataInTushare['industry'].replace([np.nan,''],'None', inplace=True)

print("The shape of the data frame: {}".format(rawCompanyDataInTushare.shape))
print("The types in the data frame {}".format(rawCompanyDataInTushare.dtypes))
rawCompanyDataInTushare

In [ ]:
# rawDailyDataInTushare=getDailyStockData()
# print("The shape of the data frame: {}".format(rawDailyDataInTushare.shape))
# print("The types in the data frame:\n{}".format(rawDailyDataInTushare.dtypes))
# print("The unique stock code: {}".format(len(rawDailyDataInTushare['ts_code'].unique())))
# rawDailyDataInTushare

In [ ]:
##test api
# testDailyStockData=getDailyStockData(stock_code="000006.SZ")
# testDailyStockData

In [ ]:
###由于Toshare社区限制每次查询只能拿到 6000 行日结交易信息（每日每公司为一行），而且每分钟只能查500次。为了得到当前所有上市公司，每次查询只查一家上市公司而且将会延迟0.2秒
def bindDailyAndCompanyInformation():
    notIncludedData=['name']

    startTime=datetime.now()
    companies=getStockCompanyInformation()
    bindData=pd.DataFrame()
    i=2
    for company in companies['ts_code'].to_numpy():
        dailyData=getDailyStockData(stock_code=company)
        for header in companies.columns:
            if header!="ts_code":
                #test only
                # print("------------------------------------")
                # print(companies[companies['ts_code'] == company][header].values.repeat(5))
                # print(dailyData.shape[0])
                # print("------------------------------------")
                dailyData[header]= companies[companies['ts_code'] == company][header].values.repeat(dailyData.shape[0])

        dailyData=dailyData.drop(columns=notIncludedData)
        bindData=bindData.append(dailyData)
        #time.sleep(0.15)
        ###########test only
        #print(dailyData)
        # if i==1:
        #     break
        # else:
        #     i-=1
        ###########
    timeConsume=datetime.now()-startTime
    print("Run time: {}".format(timeConsume))
    return bindData


In [ ]:
###此cell仅用于下载股票数据，由于信息量庞大，可导入下好的 TushareRawData.csv 文件即可
# theBindDataInTushare=bindDailyAndCompanyInformation()
# print("The shape of the data frame: {}".format(theBindDataInTushare.shape))
# print("The types in the data frame:\n{}".format(theBindDataInTushare.dtypes))
# print("The unique stock code: {}".format(len(theBindDataInTushare['ts_code'].unique())))
# theBindDataInTushare.to_csv('stockData/TushareRawData.csv', header=theBindDataInTushare.columns, index=False)

In [ ]:
###此cell是本地读取tushare股票信息，如果本地有csv文件就用这个，并且注释上一个cell，如果没下载csv文件就取消注释上一个cell
theBindDataInTushare=pd.read_csv('stockData/TushareRawData.csv')
print("The shape of the data frame: {}".format(theBindDataInTushare.shape))
print("The types in the data frame:\n{}".format(theBindDataInTushare.dtypes))
print("The unique stock code: {}".format(len(theBindDataInTushare['ts_code'].unique())))

In [ ]:
###todo 不知道为什么有100家公司查不到，手动查询看看情况，然后手动合并吧
missedCompany=[]
bindCompany=theBindDataInTushare['ts_code'].unique()
for code in rawCompanyDataInTushare['ts_code'].values:
    if code not in bindCompany:
        missedCompany.append(code)


# for code in missedCompany:
#     print("-------------------------------------")
#     print(getDailyStockData(stock_code=code,end="20220505"))
#     print(getStockCompanyInformation(stock_code=code))
#     print("-------------------------------------")

print("The num of miss company: {}".format(len(missedCompany)))

###todo 这些股票查不到，这些公司基本都是2023年上市的，可能数据还没更新吧

In [ ]:
print("The number of industry: {}".format(len(theBindDataInTushare['industry'].unique())))
print("The exact number of company: {}".format(len(theBindDataInTushare['ts_code'].unique())))
# print("--------------------------------------------------------\n"+"The industries: {}".format(theBindDataInTushare['industry'].unique()))

theCounter={}
uniqueList=rawCompanyDataInTushare['industry'].values
for industry in uniqueList:
    if industry in theCounter:
        theCounter[industry]+=1
    else:
        theCounter[industry]=1
theCounter=sorted(theCounter.items(), key = lambda kv:kv[1],reverse=True)
print("--------------------------------------------------------\n"+"the number of company in different industry:\n {}".format(theCounter))

### test science data

In [ ]:
TCLStock=theBindDataInTushare[theBindDataInTushare['ts_code']=="000100.SZ"].sort_values(by='trade_date',ascending=True)
plt.figure(figsize=(12,8))
ax = plt.gca()
locator = mdates.DayLocator(interval=30*3)
ax.xaxis.set_major_locator(locator)
plt.gcf().autofmt_xdate()
plt.plot(TCLStock['trade_date'],TCLStock['high'],label="high")
plt.plot(TCLStock['trade_date'],TCLStock['low'],label="low")
plt.xlabel("Date")
plt.ylabel("Amount")
plt.title('TCL100')
plt.legend()   #打上标签
plt.show()
TCLStock

### 1.1.1 大盘整体趋势

In [ ]:
theTrendResult=theBindDataInTushare.groupby('trade_date')[['close','open']].mean()

plt.figure(figsize=(16,8))
ax = plt.gca()
locator = mdates.DayLocator(interval=30*3)
ax.xaxis.set_major_locator(locator)
plt.gcf().autofmt_xdate()
plt.plot(theTrendResult['open'],label="open")
plt.plot(theTrendResult['close'],label="close")
plt.xlabel("Date")
plt.ylabel("Amount (RMB)")
plt.title('The Trend')
plt.legend()   #打上标签
plt.show()

theTrendResult

### 1.1.2 各个行业领域大盘

注意: 该cell运行会绘制大量的图表，图表都被保存好了，如果不想运行可以查看路径stockData/tableForIndustry/raw

In [ ]:
###生成各个领域的平均总览图标
#
# for i in list(theIndustryGroupedTrendResult):
#     plt.figure(figsize=(16,8))
#     x = np.arange(0, 300, 20)
#     plt.yticks(x)
#     i[-1]['close'].plot()
#     plt.xlabel("Date")
#     plt.ylabel("Amount (RMB)")
#     plt.title("The Industry Trend: {}".format(i[-1]['industry'].unique()[0]))
#     plt.legend()   #打上标签
#
#     ###下载图表
#     # plt.savefig("stockData/tableForIndustry/raw/industryTablesFor{}.jpg".format(format(i[-1]['industry'].unique()[0])))
#
# plt.show()
#
# theIndustryTrendResult

### 1.1.3 数据异常处理 (industry 为 none，建议优化)
注意: 行业总共110种，很多，在模型里面本身是建议降维的; 而且有些行业重复，例如金属被分成好几个领域，但是不建议归类，因为这些行业的走势都不一样，建议保留，等后续讨论。

目前计划拿到这些公司的产品与介绍，nle分析并将其分类到合适的种类

目前有两个方向，一个根据tf-idf来量化词语，一个是通过word2vec来量化(此方案为back up，原因不够精确)

In [ ]:
### 目前已知的行业有110种，还有一种没有归类，现在尝试归类
print("the number of company with none industry: {}".format(len(rawCompanyDataInTushare[rawCompanyDataInTushare['industry'] == 'None']['ts_code'].values)))


# ##查询行业缺失的公司简介与产品名称
# noneIndustryCodeList=rawCompanyDataInTushare[rawCompanyDataInTushare['industry'] == 'None']['ts_code'].values
# noneIndustryDF=pd.DataFrame()
# now=datetime.now()
# for code in noneIndustryCodeList:
#     ###该接口每分钟最多10次，尼玛得搞16分钟，之后会存在stockData/CompanyIntroductionWithNoneIndustry.csv
#     noneIndustrydf = pro.stock_company(ts_code=code, fields='ts_code,introduction,main_business')
#     noneIndustryDF=pd.concat([noneIndustryDF,noneIndustrydf])
#     time.sleep(6)
#
# noneIndustryDF.to_csv("stockData/CompanyIntroductionWithNoneIndustry.csv")
# print("cost time: {}".format(datetime.now()-now))
#
#
#
# ##查询行业已知的公司简介与产品名称
# knownIndustryCodeList=rawCompanyDataInTushare[rawCompanyDataInTushare['industry'] != 'None']['ts_code'].values
# knownIndustryDF=pd.DataFrame()
# now=datetime.now()
# for code in knownIndustryCodeList:
#     ###该接口每分钟最多10次，尼玛得搞16分钟，之后会存在stockData/CompanyIntroductionWithNoneIndustry.csv
#     knownIndustrydf = pro.stock_company(ts_code=code, fields='ts_code,introduction,main_business')
#     knownIndustryDF=pd.concat([knownIndustryDF,knownIndustrydf])
#     time.sleep(6)
#
# knownIndustryDF.to_csv("stockData/CompanyIntroductionWithKnownIndustry.csv")
# print("cost time: {}".format(datetime.now()-now))


noneIndustryDF=pd.read_csv("stockData/CompanyIntroductionWithNoneIndustry.csv", index_col=False)
knownIndustryDF=pd.read_csv("stockData/CompanyIntroductionWithKnownIndustry.csv", index_col=False)

In [ ]:
noneIndustryDF['industry']=noneIndustryDF.apply(lambda x:rawCompanyDataInTushare[x['ts_code']==rawCompanyDataInTushare['ts_code']]['industry'].values[0],axis=1)

noneIndustryDF

In [ ]:
knownIndustryDF['industry']=knownIndustryDF.apply(lambda x:rawCompanyDataInTushare[x['ts_code']==rawCompanyDataInTushare['ts_code']]['industry'].values[0],axis=1)
knownIndustryDF=knownIndustryDF[knownIndustryDF['industry']!='None']
print("行业数量（不包括None）: {}\n行业:{}".format(len(knownIndustryDF['industry'].unique()),knownIndustryDF['industry'].unique()))
knownIndustryDF

In [ ]:
### 由于有110个行业，有的重复度比较高，我打算降维一下
def decreseIndustrDiamantion(theData,name):
    theDecreased={'钢材':['特种钢','钢加工','普钢'],'酒类':['白酒','啤酒','红黄酒'],'矿业':['铝锌','铜','铝','黄金','小金属'],'石油':['石油加工','石油贸易','石油开采'],'电力':['火力发电','新型电力'],'机械':['轻工机械','纺织机械','工程机械','专用机械','机械基件'],'运输':['水运','空运','仓储物流'],'地产':['全国地产','区域地产']}
    for thisIndex, thisRow in theData.iterrows():
        for key, value in theDecreased.items():
            if thisRow[name] in value:
                thisRow[name]=key
                theData.loc[thisIndex]=thisRow
    return theData

In [ ]:
import math
###分词
import jieba
import re
import jieba.analyse as analyse

for industry in knownIndustryDF['industry'].unique():
    jieba.add_word(industry, freq=None, tag=None)


def getStopWords(url):
    with open(url,'r',encoding='utf8') as f:
        stopword = f.readlines()
    return [i.replace('\n','') for i in stopword]

def tokenizPara(text,stopword,fin_length=0.6):
    text=text.replace("."," ")
    URL_REGEX = re.compile(
    r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))',
    re.IGNORECASE)
    text = re.sub(URL_REGEX, "", text)
    text = re.sub(r"\s+", " ", text)
    sentences=re.split('，|_|-|!|。|,|/|：|:',text.strip())
    cutSentences=[]

    ###先分词
    for sentence in sentences:
        ex="///".join(jieba.cut(sentence,cut_all=False))
        cutSentences+=ex.split('///')

    ###先去除停用词与一些数字，拿到提纯的文本
    afterStop=[i for i in cutSentences if len(i)>1 and not i.isdigit() and i not in stopword]

    ###二次提纯，根据比例提出tf-idf分数高于平均的词语，并且词性作筛选,并且至少留住50%个词
    afterTfidf=dict(analyse.extract_tags(" ".join(afterStop), withWeight=True, allowPOS=('ns', 'n', 'vn')))
    averTfidf=np.mean(list(afterTfidf.values()))
    if len([k for k, v in afterTfidf.items() if v>=averTfidf]) >=len(afterStop)*fin_length:
        # print([k for k, v in afterTfidf.items() if v>=averTfidf])
        return [k for k, v in afterTfidf.items() if v>=averTfidf]
    else:
        afterTfidf=dict(sorted(afterTfidf.items(), key = lambda kv:kv[1],reverse=True))
        # print(list(afterTfidf.keys())[:math.ceil(len(afterStop)*fin_length)])
        return list(afterTfidf.keys())[:math.ceil(len(afterStop)*fin_length)]
    # return [k for k, v in afterTfidf.items()]

In [ ]:
###todo 加载stopword
cn_stopwords=getStopWords("NLP/stopwords/cn_stopwords.txt")
baidu_stopword=getStopWords("NLP/stopwords/baidu_stopwords.txt")
hit_stopword=getStopWords("NLP/stopwords/hit_stopwords.txt")
scu_stopword=getStopWords("NLP/stopwords/scu_stopwords.txt")
myStopword=['一家','','目前','公司','产品','企业','生产','山东','上市','坚实','每天','天天','完全','没有','众多','提供','完全','以古','地方','典当','拥有','下属','固有','无处不在','行业','销售','业务','主要','主要业务']
limitedIndustryList=['机场','批发业','陶瓷','商品城','林业','电器连锁','公路']
stopword_list=cn_stopwords+hit_stopword+baidu_stopword+scu_stopword+myStopword
for stopword in stopword_list:
    jieba.add_word(stopword, freq=None, tag=None)

In [ ]:
###todo 开始分词
unknownIndustryParagraphs=[]
unknownIndustryCutKeyword=[]
for item in noneIndustryDF.values:
    # item2=""
    # if item[2]!=np.nan and item[2]!=np.NAN and item[2]!="nan":
    #     item2=item[2]
    tokens=tokenizPara(str(item[1])+str(item[2]),stopword_list)
    unknownIndustryParagraphs.append(tokens)
    unknownIndustryCutKeyword.append(" ".join(tokens))

knownIndustryParagraphs=[]
knownIndustryCutKeyword=[]
for item in knownIndustryDF.values:
    # item2=""
    # if item[2]!=np.nan and item[2]!=np.NAN and item[2]!="nan":
    #     item2=item[2]
    tokens=tokenizPara(str(item[1])+str(item[2]),stopword_list)
    knownIndustryParagraphs.append(tokens)
    knownIndustryCutKeyword.append(" ".join(tokens))


# print("------------------------------------\n"
#       "unknowIndustry:\n{}"
#       "\n------------------------------------\n"
#       "knowIndustry:\n{}".format(unknownIndustryParagraphs[:3],knownIndustryParagraphs[:3]))

knownIndustryDF['keyword']=knownIndustryParagraphs
knownIndustryDF['cut_keywords']=knownIndustryCutKeyword
noneIndustryDF['keyword']=unknownIndustryParagraphs
noneIndustryDF['cut_keywords']=unknownIndustryCutKeyword

In [ ]:
###todo test jieba
print("原文：",knownIndustryDF['keyword'].values[0])
testwordlist=" ".join(knownIndustryDF['keyword'].values[0])
keywords = analyse.extract_tags(testwordlist, topK=5, withWeight=False, allowPOS=('ns', 'n', 'vn'))
print("tf-idf筛选后（最高的5个词语）：",keywords)

In [ ]:
###todo 行业降维
knownIndustryDF=decreseIndustrDiamantion(knownIndustryDF,'industry')
knownIndustryDF

In [ ]:
### todo 分类并合并数据
# print(knownIndustryDF.head())
tempdf=knownIndustryDF
###根据行业合并文本，合并材料
introductionByIndustry=pd.DataFrame()
for intro in list(tempdf.groupby('industry')):
    paras=[]
    AllIntroduction=[]
    for para in intro[1][['keyword','introduction']].values:
        paras+=para[0]
        AllIntroduction.append(para[1])

    ###calculate word frequency
    freq={}
    for word in paras:
        if word in freq:
            freq[word]+=1
        else:
            freq[word]=1
    freq=sorted(freq.items(), key = lambda kv:kv[1],reverse=True)
    freq = dict(freq)
    #('机场', 5), ('批发业', 5), ('陶瓷', 5), ('商品城', 4), ('林业', 4), ('电器连锁', 2), ('公路', 2)
    ###开始排除部分词频为1的词语，减少影响，以上的行业关键词看情况减少，因为样本数量实在太少了
    notIncludedIndustryWords=[]
    # for key, value in freq.items():
    #     if value==1:
    #         notIncludedIndustryWords.append(key)
    #         stopword_list.append(key)
    # if intro[0] not in limitedIndustryList:
    # print("industry: {}, notIncl: {}".format(intro[0],notIncludedIndustryWords))
    # paras=[x for x in paras if x not in notIncludedIndustryWords]
    row=pd.DataFrame({'industry':intro[0],'keyword':[paras],'AllIntro':[AllIntroduction],'frequency':[freq]})
    introductionByIndustry=pd.concat([row,introductionByIndustry])

introductionByIndustry['cut_keywords']=introductionByIndustry.apply(lambda x:" ".join(x['keyword']), axis=1)

###
#对未知行业的关键词也进行筛选
noneIndustryDF['keyword']=noneIndustryDF['keyword'].apply(lambda x: [j for j in x if j not in stopword_list])
knownIndustryDF['keyword']=knownIndustryDF['keyword'].apply(lambda x : [j for j in x if j not in stopword_list])
introductionByIndustry

In [ ]:
introductionByIndustry.loc[introductionByIndustry["industry"].isin(limitedIndustryList)]

#### 公司行业归类(tf-idf) (废弃，该算法已融入进下一个方案)

In [ ]:
###量化词语成tfidf分数
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer,CountVectorizer
corpus_list=knownIndustryDF['cut_keywords'].values.tolist()+noneIndustryDF['cut_keywords'].values.tolist()
vectorizer = TfidfVectorizer(use_idf=True, smooth_idf=True, norm=None)
tfidf = vectorizer.fit_transform(corpus_list)
weight = tfidf.toarray()
vocab = vectorizer.get_feature_names_out()
print("\n词汇表大小：{}\n{}".format(len(vocab),vocab))
print("\n权重形状：{}\n{}".format(weight.shape,weight))
weight

#### 公司行业归类(word2vec词向量)

In [ ]:
from gensim.models import KeyedVectors, word2vec
# ### todo 训练词向量库
# sentences=knownIndustryDF['keyword'].values.tolist()+noneIndustryDF['keyword'].values.tolist()
# model = word2vec.Word2Vec(sentences,min_count=1, vector_size=200,sg=1)
#
# ### todo 保存词向量库
# model.wv.save('NLP/wordVector/MineWordVec/MyWordModel.model')

In [ ]:
###todo 准备词向量库
###100向量
# model = KeyedVectors.load_word2vec_format('NLP/wordVector/tencent-ailab-embedding-zh-d100-v0.2.0/tencent-ailab-embedding-zh-d100-v0.2.0.txt', binary=False)

###200向量
model = KeyedVectors.load_word2vec_format('NLP/wordVector/tencent-ailab-embedding-zh-d200-v0.2.0/tencent-ailab-embedding-zh-d200-v0.2.0.txt', binary=False)

###自己的词库，不怎么样
# model=KeyedVectors.load('NLP/wordVector/MineWordVec/MyWordModel.model')

In [ ]:
###todo 把词库里没有的词去掉
noneIndustryDF['keyword']=noneIndustryDF['keyword'].apply(lambda x: [j for j in x if j in model])
knownIndustryDF['keyword']=knownIndustryDF['keyword'].apply(lambda x : [j for j in x if j in model])
introductionByIndustry['keyword']=introductionByIndustry['keyword'].apply(lambda x : [j for j in x if j in model])
###todo 去掉不相干词语
# knownIndustryDF['keyword']=knownIndustryDF['keyword'].apply(lambda x:[j for j in x if j!=model.doesnt_match(x)])
# introductionByIndustry['keyword']=introductionByIndustry['keyword'].apply(lambda x:[j for j in x if j!=model.doesnt_match(x)])
# noneIndustryDF['keyword']=noneIndustryDF['keyword'].apply(lambda x:[j for j in x if j!=model.doesnt_match(x)])

In [ ]:
largestWordInKnow=0
for paragra in knownIndustryDF['keyword'].values:
    if len(paragra)>largestWordInKnow:
        largestWordInKnow=len(paragra)

largestWordInUnknow=0
for paragra in noneIndustryDF['keyword'].values:
    if len(paragra)>largestWordInUnknow:
        largestWordInUnknow=len(paragra)

largestWordInIntro=0
for paragra in introductionByIndustry['keyword'].values:
    if len(paragra)>largestWordInIntro:
        largestWordInIntro=len(paragra)

print("The largest num of word in Known: {}".format(largestWordInKnow))
print("The largest num of word in unKnown: {}".format(largestWordInUnknow))
print("The largest num of word in Introduction: {}".format(largestWordInIntro))

In [ ]:
##["我不看好当前股票市场"] negative -1.0 -4.0
###["我对当前市场持积极股票"] positive 2.0 5.0
###["我保持观望态度"] neutral -1 1


###wordbag:['失望':-0.7,.....]

##### 方案一: 直接调用接口（目前最优解）

In [ ]:
###todo直接测试句向量similarity,看看准确率
###这个cell的对比方法是[所有相同行业的公司简介分词]:[当前某个None行业公司简介分词]
def getWordVector(keywords=None,w2v_model=model):
    results={}
    for index, row in introductionByIndustry.iterrows():
        results[row['industry']]=w2v_model.n_similarity(keywords,row['keyword'])
    results=sorted(results.items(), key = lambda kv:kv[1],reverse=True)
    return results


print("第一种方式:\n")
i=0
for index, row in noneIndustryDF.iterrows():
    if i==5:
        break
    else:
        i+=1

    print("------------------------------------")
    print("第{}原文: {}".format(i,row['introduction']))
    results=getWordVector(keywords=row['keyword'])
    # print("原分类:")
    # print(row['industry'])
    print("相似度:")
    print(results[:10])

# correct_num=0
# for index, row in knownIndustryDF.iterrows():
#     results=dict(getWordVector(keywords=row['keyword']))
#     if next(iter(results.keys()))==row['industry']:
#         correct_num+=1
# print("accuracy score: {}%".format((correct_num/len(knownIndustryDF['industry'].values)*100)))
###腾讯词库准确率:52.54%
###自己训练的词库: 35.38%

###200向量的准确率 56.69%

In [ ]:
###这个cell的对比方法是[单个行业的某个公司简介分词]:[当前某个None行业公司简介分词]
###最后取平均值
print("第二种方式:\n")
i=0
for index, row in noneIndustryDF.iterrows():
    if i==5:
        break
    else:
        i+=1

    print("第{}原文: {}".format(i,row['introduction']))

    meanScore={}
    for intro in list(knownIndustryDF.groupby('industry')):
        scores=[]
        for para in intro[1][['industry','keyword']].values:
            scores.append(model.n_similarity(para[0],row['keyword']))
        meanScore[intro[1]['industry'].values[0]]=sum(scores)/len(scores)
    meanScore=sorted(meanScore.items(),key=lambda k:k[1],reverse=True)
    print("相似度: \n{}".format(meanScore[:5]))

##### 方案二: Xgboost
上面两个cell的结果是直接用句向量数值进行比较，得出每个行业的相似度，相似度越高，与这个行业契合度越高，然后每个cell都有一种方法(在注释)。可是经过测试证明，第二个cell采取平均值完全没用，第一个cell虽然能有效地把公司简介主题归化从110个到5-2个左右，但是到了这个范围，准确率就不高了，只能说取得了初步效果，但是不是很理想

第二次测试一下神经网络与xgboost，不过需要把每个词都改成向量，而不是句向量

一开始直接把similarity输进去训练，准确率只有46，方法不对，不如直接调用word2vec接口


In [ ]:
###todo 准备训练输入数据
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb

def average_word_vectors(words,model,vocabulary,num_features):
    feature_vector=np.zeros((num_features,),dtype='float64')
    nwords=0
    for word in words:
        if word in vocabulary:
            nwords=nwords+1
            feature_vector=np.add(feature_vector,model[word])
    if nwords:
        feature_vector=np.divide(feature_vector,nwords)
    return feature_vector

###a: 145word word:200 vector
###b: 155wod word:200 vect
###a:[] shape(155)   b:[] shape(155)
###a sum[vector]/word_count
### 200
###

# def average_word_vectors(words,model,vocabulary,num_features):
#     feature_vector=np.zeros((num_features,),dtype='float64')
#     n_words=0
#     for word in words:
#         feature_vector[n_words]=round(model[word].mean(),5)
#         n_words+=1
#         if n_words==num_features:
#             break
#     return feature_vector

def averaged_word_vectorizer(corpus,model,num_features):
    #get the all vocabulary
    vocabulary=model
    features=[average_word_vectors(tokenized_sentence,model,vocabulary,num_features) for tokenized_sentence in corpus]
    return np.array(features)

# def get_word_vectors(data,model):
#     words_art=[]
#     for i in range(len(data)):
#         words_art.append(eval(data.loc[i]))
#     return averaged_word_vectorizer(words_art,model,num_features=300)

###todo 词向量模型分析,准备数据输入
### 量化keyword
knownIndustryDF['keyword_vector']=knownIndustryDF['keyword'].apply(lambda x:[model.get_vector(j) for j in x])
# knownIndustryDF['keyword_vector_mean']=knownIndustryDF['keyword'].apply(lambda x : [model.get_vector(i).mean() for i in x])
# knownIndustryDF['keyword_length']=knownIndustryDF['keyword'].apply(lambda x: len(x))
# knownIndustryDF['sentence_vector_sum']=knownIndustryDF['keyword_vector'].apply(lambda x : sum(x))
# knownIndustryDF['sentence_vector_mean']=knownIndustryDF['keyword_vector'].apply(lambda x : sum(x)/len(x))
# ###把句向量也引进来
# for index, row in introductionByIndustry.iterrows():
#     knownIndustryDF['n_similarity_'+row['industry']]=knownIndustryDF['keyword'].apply(lambda x: model.n_similarity(x,row['keyword']))

# TheDataSet=pd.read_csv("stockData/keywordVecIntroductionWithKnownIndustry.csv")
# TheDataSet=TheDataSet.drop(labels=['keyword_vector','keyword_vector_mean','industry','main_business','introduction','ts_code','keyword'], axis=1)

TheDataSet=knownIndustryDF
x=averaged_word_vectorizer(knownIndustryDF['keyword'].values,model,model.vector_size)


###one-hot
encoder = OneHotEncoder(sparse=False)
y=encoder.fit_transform([[x] for x in TheDataSet['industry'].values])
# x=TheDataSet
###分割数据集
x_train , x_test , y_train , y_test = train_test_split(x,y, test_size= 0.1 ,random_state=42)

In [ ]:
###准备参数,开始训练
# param_grid = {
#             'max_depth': [2, 3, 4, 5, 6, 7, 8],
#             'n_estimators': [30, 50, 100, 300, 500, 1000,2000],
#             'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.01, 0.02, 0.03, 0.05, 0.5],
#             "gamma":[0.0, 0.1, 0.2, 0.3, 0.4],
#             "reg_alpha":[0.0001,0.001, 0.01, 0.1, 1, 100],
#             "reg_lambda":[0.0001,0.001, 0.01, 0.1, 1, 100],
#             "min_child_weight": [2,3,4,5,6,7,8],
#             "colsample_bytree": [0.6, 0.7, 0.8, 0.9],
#             "subsample":[0.6, 0.7, 0.8, 0.9]
# }
param_grid = {
            'max_depth': [2, 4, 6, 8],
            'n_estimators': [50, 100, 300, 500,1000],
            "gamma":[0.1, 0.2, 0.3],
            "min_child_weight": [2,4,6,8],
}

# other_params = {'learning_rate': 0.3, 'n_estimators': 400, 'max_depth': 5, 'min_child_weight': 1, 'seed': 42, 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}
# grid_xgboost = GridSearchCV(estimator=xgb.XGBRegressor(seed=27), param_grid=param_grid, cv=5)
# multioutputregressor = MultiOutputRegressor(grid_xgboost).fit(x_train, y_train)

multioutputregressor = MultiOutputRegressor(xgb.XGBRegressor(max_depth=5,min_child_weight=3,learning_rate=0.1,n_estimators=400,objective='binary:logistic')).fit(x_train, y_train)

In [ ]:
### 熟料 发贴    [熟料 发贴]
###熟料          [1,0]
###发贴          [0,1]

In [ ]:
###todo 验证
from sklearn import metrics
from sklearn.metrics import classification_report
# def get_scores(preds,gt):
#     # print ('AUC: %.4f' % metrics.roc_auc_score(gt,preds))
#     print ('ACC: %.4f' % metrics.accuracy_score(gt,preds))
#     print('macro')
#     print( 'Recall: %.4f' % metrics.recall_score(y_test,preds,average='macro'))
#     print( 'F1-score: %.4f' %metrics.f1_score(gt,preds,average='macro'))
#     print( 'Precision: %.4f' %metrics.precision_score(gt,preds,average='macro'))
#
#     print('\nmicro:')
#     print( 'Recall: %.4f' % metrics.recall_score(y_test,preds,average='micro'))
#     print( 'F1-score: %.4f' %metrics.f1_score(gt,preds,average='micro'))
#     print( 'Precision: %.4f' %metrics.precision_score(gt,preds,average='micro'))

check=multioutputregressor.predict(x_test)
for i in range(len(check)):
        max_value=max(check[i])
        for j in range(len(check[i])):
            if max_value==check[i][j]:
                check[i][j]=1
            else:
                check[i][j]=0

print(classification_report(y_test, check))
mse = mean_squared_error(y_test, check)
print("mse score: {}%".format(round(mse*100,2)))

result={"result":[j[0] for j in encoder.inverse_transform(check)],"test_answer":[j[0] for j in encoder.inverse_transform(y_test)]}

result_check=pd.DataFrame(result)

accurate_num=0
for index, element in enumerate(encoder.inverse_transform(y_test)):
    if element==encoder.inverse_transform(check)[index]:
        accurate_num+=1

print("accuracy score: {}%".format(round((accurate_num/len(y_test))*100,2)))
result_check
### word2vec & xgboost 表现不是很好, mse:0.65%, 准确率 47%，不过个人认为方法完全错了

##### 方案三: svm 向量机

In [ ]:
from sklearn import svm

clf = MultiOutputRegressor(svm.SVC(C=5, gamma= 1, probability=True)).fit(x_train, y_train)
y_predict = clf.predict(x_test)
print(classification_report(y_test, y_predict))

for i in range(len(y_predict)):
        max_value=max(y_predict[i])
        for j in range(len(y_predict[i])):
            if max_value==y_predict[i][j]:
                y_predict[i][j]=1
            else:
                y_predict[i][j]=0
result={"result":[j[0] for j in encoder.inverse_transform(y_predict)],"test_answer":[j[0] for j in encoder.inverse_transform(y_test)]}

result_check=pd.DataFrame(result)

accurate_num=0
for index, element in enumerate(encoder.inverse_transform(y_test)):
    if element==encoder.inverse_transform(y_predict)[index]:
        accurate_num+=1

print("accuracy score: {}%".format(round((accurate_num/len(y_test))*100,2)))
result_check

##### 方案四: textCnn


In [ ]:
import tensorflow as tf
from keras.models import Sequential,Model
from keras.models import load_model
from keras.layers import Dense,Dropout,Activation,Input, Lambda, Reshape,concatenate
from keras.layers import Embedding,Conv1D,MaxPooling1D,GlobalMaxPooling1D,Flatten,BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical, pad_sequences


#导入word2vec模型并进行预处理
def w2v_model_preprocessing(content,w2v_model,max_len=32):
    # 初始化 `[word : index]` 字典
    word2idx = {"_PAD": 0}
    # 训练数据 词汇表构建
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(content)
    vocab_size = len(tokenizer.word_index)  # 词库大小
    print(tokenizer.word_index)
    error_count = 0
    # 存储所有 word2vec 中所有向量的数组，其中多一位，词向量全为 0， 用于 padding
    embedding_matrix = np.zeros((vocab_size + 1, w2v_model.vector_size))
    print(embedding_matrix.shape)
    for word, i in tokenizer.word_index.items():
        if word in w2v_model:
            embedding_matrix[i] = w2v_model[word]
        else:
            error_count += 1
    # 训练数据 词向量截断补全(padding)
    seq = tokenizer.texts_to_sequences(content)
    trainseq = pad_sequences(seq, maxlen=max_len,padding='post')
    return embedding_matrix,trainseq


def build_textcnn(max_len,embeddings_dim,embeddings_matrix):
    #构建textCNN模型
    main_input = Input(shape=(max_len,), dtype='float64')
    # 词嵌入（使用预训练的词向量）
    embedder = Embedding(
                         len(embeddings_matrix), #表示文本数据中词汇的取值可能数,从语料库之中保留多少个单词
                         embeddings_dim, # 嵌入单词的向量空间的大小
                         input_length=max_len, #规定长度
                         weights=[embeddings_matrix],# 输入序列的长度，也就是一次输入带有的词汇个数
                         trainable=False # 设置词向量不作为参数进行更新
                         )
    embed = embedder(main_input)
    flat = Flatten()(embed)
    dense01 = Dense(5096, activation='relu')(flat)
    dense02 = Dense(1024, activation='relu')(dense01)
    main_output = Dense(2, activation='softmax')(dense02)
    model = Model(inputs=main_input, outputs=main_output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

embedding_matrix,trainSet=w2v_model_preprocessing(knownIndustryDF['keyword'].values,model)
embedding_matrix

如果写到这一步，证明主题分类的模型已经写好了，虽然不知道质量怎么样hhh。其实说真的，缺失行业标记的公司不过才164家，人力做的话几天就能写完了，不过我没做过中文的词义分析，这算是一个nlp的开始吧，但是说真的太麻烦了，希望以后有高效的方法，

In [ ]:
### todo 泛化


## 1.2 use yfinance (雅虎非官方API) api get stock data

In [ ]:
#测试api
data = yf.download(tickers='AMD',start='2013-01-1',end='2023-01-01')
print("The types in the data frame:\n{}".format(data.dtypes))
data

In [ ]:
amd = yf.Ticker("AMD")
info=pd.DataFrame.from_dict(amd.info)
print("The types in the data frame:\n{}".format(info.dtypes))
info

In [ ]:
data["Close"].plot()
plt.show()

In [ ]:
##获取标普500股票公司名字
import bs4 as bs
import requests

###爬取维基百科标普500公司名称
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

###筛选元素
tickers = []
for row in table.findAll('tr')[1:]:
   ticker = row.findAll('td')[0].text
   ticker=ticker.replace("\n","")
   tickers.append(ticker)


tickers

In [ ]:
def getStockDataInYFinance(name,start='2013-01-1',end='2023-01-01'):
    includedList=['country','zip','industry','sector']
    data = yf.download(tickers=name,start=start,end=end)
    data['trade_date'] = data.index
    company = yf.Ticker(name)
    try:
        companyInfo=pd.DataFrame.from_dict(company.info)
    except BaseException:
        print(company.info)
        return np.nan
    else:
        data['name']=[name]*data.shape[0]
        for column in companyInfo.columns:
            if column in includedList:
                data[column]=np.array([companyInfo[column].values[0]]).repeat(data.shape[0])

    return data


### get all stock data and store them
def storeAllStockDataForYFinance():
    now=datetime.now()
    allStockDataInYFinance=pd.DataFrame()
    for company in tickers:
        companyInfo=getStockDataInYFinance(company)
        if companyInfo is np.nan:
            print("break, company {} data is null")
            continue
        else:
            allStockDataInYFinance=pd.concat([allStockDataInYFinance,companyInfo])
    print("Cost time: {}".format(datetime.now()-now))
    return allStockDataInYFinance

In [ ]:
getStockDataInYFinance(tickers[0]).dtypes

In [ ]:
###仅用于下载yfinance 股票数据
# stockDataInYFinance=storeAllStockDataForYFinance()
# stockDataInYFinance.to_csv('stockData/YFinanceRawData.csv', header=stockDataInYFinance.columns,index=False)

In [ ]:
###验证一下有多少个查漏了
stockDataInYFinance=pd.read_csv('stockData/YFinanceRawData.csv')
uniqueCompany=stockDataInYFinance['name'].unique()
missedList=[]
for ticker in tickers:
    if ticker not in uniqueCompany:
        missedList.append(ticker)

print("length of miss company: {}".format(len(missedList)))
###总共有3个公司遗漏

###手动查询
for company in missedList:
    print("------------------------------------------------------")
    print("This is the information for {}".format(company))
    print(getStockDataInYFinance(name=company))
    print("------------------------------------------------------")
###经过调查，属于信息不完善，有的连公司信息都不全

In [ ]:
print("types in data: {}".format(stockDataInYFinance.dtypes))
stockDataInYFinance

In [ ]:
MMMStock=stockDataInYFinance[stockDataInYFinance['name']=="MMM"]

plt.figure(figsize=(15,8))

ax = plt.gca()
locator = mdates.DayLocator(interval=30*3)
ax.xaxis.set_major_locator(locator)

x=MMMStock['trade_date']

plt.plot(x,MMMStock['High'],label="high")
plt.plot(x,MMMStock['Low'],label="low")
plt.gcf().autofmt_xdate()
plt.xlabel("Date")
plt.ylabel("Amount")
plt.title('MMM')
plt.rc('xtick', labelsize=15)
plt.rc('ytick', labelsize=15)
plt.rcParams.update({'font.size': 15})
plt.legend()   #打上标签
plt.show()
MMMStock

# 2.0 Analysis the raw data in AI
* data engineering
* data cleaning
* build model
* train

------------------------------------------
这里说一下训练集与验证集的分配:
* 训练集: 截取 2013-01-01 to 2018-01-01，5年数据，占总比50%。
* 验证集1: 截取 2018-01-01 to 2019-01-01 1年数据， 占总比10%，因为疫情就发生在2019年，该验证集用于检测模型准确度，确保该模型在疫情前时期可以正常使用
* 验证集2: 截取 2019-01-01 to 2023-01-01 4年数据，占比40%，该数据集并非用于验证，而是在确保验证集1能正常运行的情况下，用于观察疫情发生与正常预期是否有大偏差（验证是否疫情对股票市场有巨大影响）